<a href="https://colab.research.google.com/github/vggls/medical_xai/blob/main/experiments/non_medical_datasets_haas/STL_10_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [ ]:
pip install grad-cam

In [ ]:
from torchvision import transforms, datasets
import torch
import matplotlib.pyplot as plt
from torch import nn
import torch.optim as optim
from torch.utils.data import random_split
from pytorch_grad_cam import HiResCAM
import pickle
import os
from torch.utils.data import DataLoader

In [ ]:
from training_loop import Train
from train_model import fit
from testing_report import Test_Report
from haas import Haas, HA_image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Load Data**

In [ ]:
transforms = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = datasets.STL10(root='./data', split='train', folds=0, download=True, transform=transforms)
for fold in [1,2,3,4,5,6,7]:
    train_set += datasets.STL10(root='./data', split='train', folds=fold, download=True, transform=transforms)

val_set = datasets.STL10(root='./data', split='train', folds=8, download=True, transform=transforms)
val_set += datasets.STL10(root='./data', split='train', folds=9, download=True, transform=transforms)

test_set = datasets.STL10(root='./data', split='test', download=True, transform=transforms)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
len(train_set), len(val_set), len(test_set)

(8000, 2000, 8000)

In [ ]:
classes = test_set.classes
classes

['airplane',
 'bird',
 'car',
 'cat',
 'deer',
 'dog',
 'horse',
 'monkey',
 'ship',
 'truck']

In [ ]:
# class distribution - we see that the dataset is balanced
for set, name in zip([train_set, val_set, test_set], ['training set', 'validation set', 'test set']):
    print(name)
    for label in [i for i in range(10)]:
        print('label {}: instances {}'.format(label, len([(img, lbl) for (img,lbl) in set if lbl==label])))
    print('-'*30)

training set
label 0: instances 800
label 1: instances 800
label 2: instances 800
label 3: instances 800
label 4: instances 800
label 5: instances 800
label 6: instances 800
label 7: instances 800
label 8: instances 800
label 9: instances 800
------------------------------
validation set
label 0: instances 200
label 1: instances 200
label 2: instances 200
label 3: instances 200
label 4: instances 200
label 5: instances 200
label 6: instances 200
label 7: instances 200
label 8: instances 200
label 9: instances 200
------------------------------
test set
label 0: instances 800
label 1: instances 800
label 2: instances 800
label 3: instances 800
label 4: instances 800
label 5: instances 800
label 6: instances 800
label 7: instances 800
label 8: instances 800
label 9: instances 800
------------------------------


## **Custom VGG19 model**

In [ ]:
'''
# https://github.com/rasbt/deeplearning-models/blob/master/pytorch_ipynb/cnn/cnn-vgg19.ipynb
# https://blog.techcraft.org/vgg-19-convolutional-neural-network/
# https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9800759

# i.e. we must remove the 2nd 512-block in order to fit the stl10 data

from torchvision import models
vgg = models.vgg19(pretrained=True)
vgg
'''

In [ ]:
import collections
from torchvision import models
import torch.nn as nn

class VGG19_STL10():

    def __init__(self,
                 num_classes,
                 trainable_feature_layers=None):

        self.num_classes = num_classes
        self.trainable_feature_layers = trainable_feature_layers

        vgg19 = models.vgg19(pretrained=True)

        features = vgg19.features[:26] # as per paper HAAS paper FIGURE 4 (c)

        vgg19.avgpool = nn.Flatten()
        flatten = vgg19.avgpool

        classifier = nn.Linear(512 * 12 * 12, self.num_classes)

        self.model = nn.Sequential(collections.OrderedDict([
            ("features", features),
            ("flatten", flatten),
            ("classifier", classifier)]
        ))

        # LAYERS TO FREEZE DURING TRAINING
        if self.trainable_feature_layers==None:
            self.freeze = self.model.features
        else:
            assert all(x in range(len(self.model.features)) for x in self.trainable_feature_layers)
            self.freeze = [self.model.features[j] for j in range(len(self.model.features)) \
                           if j not in self.trainable_feature_layers]

## **Loop (16 models)**

In [ ]:
# Model no.4 - lr:0.0001 - wd:0.1 - bs:128 - scheduler:no

drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/STL-10/'

count = 0

accuracies, macro_f1_scores, mean_roc_auc_scores = [], [], []
haas_scores = []

for lr in [0.0001, 0.000001]:
    for wd in ['yes', 'no']:
        for bs in [32, 128]:
            for scheduler in ['yes', 'no']:

                count += 1

                vgg19 = VGG19_STL10(num_classes=10,
                              trainable_feature_layers=[i for i in range(19, 26)])

                loss_fct = torch.nn.CrossEntropyLoss()

                if wd == 'yes':
                  optimizer = optim.Adam(vgg19.model.parameters(), lr=lr, weight_decay=0.1)
                else:
                  optimizer = optim.Adam(vgg19.model.parameters(), lr=lr)

                if scheduler == 'yes':
                  sched = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
                else:
                  sched = None

                print('\033[1m' + 'Model no.{} - lr:{} - wd:{} - bs:{} - scheduler:{}'.format(count, lr, wd, bs, scheduler) + '\033[0m')

                training_dict, validation_dict = fit(train_dataset=train_set,
                                                    validation_dataset=val_set,
                                                    batch_size=bs,
                                                    model=vgg19,
                                                    loss_fct=loss_fct,
                                                    optimizer=optimizer,
                                                    scheduler=sched,
                                                    epochs=10,
                                                    patience=None,
                                                    no_of_classes=10,
                                                    labels_of_normal_classes=None)

                model = torch.load('model_epoch10.pt')
                model = model.cuda()
                model = model.eval()

                # MODEL CLASSIFICATION METRICS
                test_loader = DataLoader(dataset=test_set, batch_size=128, shuffle=False)
                test_report = Test_Report(test_loader, model, classes)

                accuracies.append(test_report.accuracy())

                macro_f1_scores.append(test_report.f1('macro'))

                model_roc_auc_scores = test_report.roc_curve_and_scores(False)
                mean_roc_auc_scores.append(sum(model_roc_auc_scores)/len(model_roc_auc_scores))

                # MODEL HAAS SCORE
                print('HiResCAM - HAAS calculation')
                hirescam_instance = HiResCAM(model=model, target_layers=[model.features[-1]], use_cuda=True)
                hirescam_score, _, _, _ = Haas(test_set, model, hirescam_instance)
                haas_scores.append(hirescam_score)
                print('HAAS score:', hirescam_score)

                if hirescam_score>1:
                  torch.save(model, drive_path + 'stl10_vgg19_no{}.pt'.format(count))
                  with open(drive_path + 'stl10_vgg19_no{}_training_dict.pickle'.format(count), 'wb') as f:
                    pickle.dump(training_dict, f)
                  with open(drive_path + 'stl10_vgg19_no{}_validation_dict.pickle'.format(count), 'wb') as f:
                    pickle.dump(validation_dict, f)

                os.remove('model_epoch10.pt')

                print('*'*100)
                print('\n')

with open(drive_path + 'stl10_haas_scores.pickle', 'wb') as f: pickle.dump(haas_scores, f)
with open(drive_path + 'stl10_accuracies.pickle', 'wb') as f: pickle.dump(accuracies, f)
with open(drive_path + 'stl10_macro_f1_scores.pickle', 'wb') as f: pickle.dump(macro_f1_scores, f)
with open(drive_path + 'stl10_mean_roc_auc_scores.pickle', 'wb') as f: pickle.dump(mean_roc_auc_scores, f)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.1 - lr:0.0001 - wd:yes - bs:32 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  lr value 0.0001
  Loss=1.2947
  Accuracy=0.67
->Validation phase
  Loss=0.4344
  Accuracy=0.86
-----------------------------------
Epoch 2/10
->Training phase
  lr value 0.0001
  Loss=0.5458
  Accuracy=0.81
->Validation phase
  Loss=0.6117
  Accuracy=0.79
-----------------------------------
Epoch 3/10
->Training phase
  lr value 5e-05
  Loss=0.5148
  Accuracy=0.82
->Validation phase
  Loss=0.6383
  Accuracy=0.78
-----------------------------------
Epoch 4/10
->Training phase
  lr value 5e-05
  Loss=0.5934
  Accuracy=0.8
->Validation phase
  Loss=0.6394
  Accuracy=0.78
-----------------------------------
Epoch 5/10
->Training phase
  lr value 2.5e-05
  Loss=0.5446
  Accuracy=0.82
->Validation phase
  Loss=0.6656
  Accuracy=0.77
-----------------------------------
Epoch 6/10
->Training phase
  lr value 2.5e-05
  Loss=0.5633
  Accurac

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.2 - lr:0.0001 - wd:yes - bs:32 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=1.3399
  Accuracy=0.66
->Validation phase
  Loss=0.4508
  Accuracy=0.86
-----------------------------------
Epoch 2/10
->Training phase
  Loss=0.5240
  Accuracy=0.82
->Validation phase
  Loss=0.7724
  Accuracy=0.74
-----------------------------------
Epoch 3/10
->Training phase
  Loss=0.7291
  Accuracy=0.75
->Validation phase
  Loss=0.7822
  Accuracy=0.73
-----------------------------------
Epoch 4/10
->Training phase
  Loss=0.8264
  Accuracy=0.72
->Validation phase
  Loss=0.8324
  Accuracy=0.71
-----------------------------------
Epoch 5/10
->Training phase
  Loss=0.9414
  Accuracy=0.67
->Validation phase
  Loss=0.9234
  Accuracy=0.68
-----------------------------------
Epoch 6/10
->Training phase
  Loss=0.9563
  Accuracy=0.66
->Validation phase
  Loss=1.0218
  Accuracy=0.66
-----------------------------------
Epoch 7/10
->Tra

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



...

Model no.14 - lr:1e-06 - wd:no - bs:32 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=2.7482
  Accuracy=0.31
->Validation phase
  Loss=1.5814
  Accuracy=0.5
-----------------------------------
Epoch 2/10
->Training phase
  Loss=1.2087
  Accuracy=0.59
->Validation phase
  Loss=1.0449
  Accuracy=0.65
-----------------------------------
Epoch 3/10
->Training phase
  Loss=0.8168
  Accuracy=0.72
->Validation phase
  Loss=0.8156
  Accuracy=0.72
-----------------------------------
Epoch 4/10
->Training phase
  Loss=0.6099
  Accuracy=0.79
->Validation phase
  Loss=0.6605
  Accuracy=0.78
-----------------------------------
Epoch 5/10
->Training phase
  Loss=0.4679
  Accuracy=0.85
->Validation phase
  Loss=0.5593
  Accuracy=0.81
-----------------------------------
Epoch 6/10
->Training phase
  Loss=0.3703
  Accuracy=0.88
->Validation phase
  Loss=0.4774
  Accuracy=0.85
-----------------------------------
Epoch 7/10
->Train

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.15 - lr:1e-06 - wd:no - bs:128 - scheduler:yes
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  lr value 1e-06
  Loss=3.8709
  Accuracy=0.19
->Validation phase
  Loss=2.7452
  Accuracy=0.29
-----------------------------------
Epoch 2/10
->Training phase
  lr value 1e-06
  Loss=2.1975
  Accuracy=0.38
->Validation phase
  Loss=1.9343
  Accuracy=0.42
-----------------------------------
Epoch 3/10
->Training phase
  lr value 5e-07
  Loss=1.7002
  Accuracy=0.48
->Validation phase
  Loss=1.7164
  Accuracy=0.47
-----------------------------------
Epoch 4/10
->Training phase
  lr value 5e-07
  Loss=1.4956
  Accuracy=0.53
->Validation phase
  Loss=1.5401
  Accuracy=0.52
-----------------------------------
Epoch 5/10
->Training phase
  lr value 2.5e-07
  Loss=1.3601
  Accuracy=0.57
->Validation phase
  Loss=1.4653
  Accuracy=0.54
-----------------------------------
Epoch 6/10
->Training phase
  lr value 2.5e-07
  Loss=1.2838
  Accuracy

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.



Model no.16 - lr:1e-06 - wd:no - bs:128 - scheduler:no
Device: cuda:0
Starting training..
-----------------------------------
Epoch 1/10
->Training phase
  Loss=3.9833
  Accuracy=0.19
->Validation phase
  Loss=2.6808
  Accuracy=0.29
-----------------------------------
Epoch 2/10
->Training phase
  Loss=2.1893
  Accuracy=0.37
->Validation phase
  Loss=1.8603
  Accuracy=0.44
-----------------------------------
Epoch 3/10
->Training phase
  Loss=1.6059
  Accuracy=0.49
->Validation phase
  Loss=1.4561
  Accuracy=0.52
-----------------------------------
Epoch 4/10
->Training phase
  Loss=1.2725
  Accuracy=0.58
->Validation phase
  Loss=1.2207
  Accuracy=0.58
-----------------------------------
Epoch 5/10
->Training phase
  Loss=1.0555
  Accuracy=0.64
->Validation phase
  Loss=1.0490
  Accuracy=0.65
-----------------------------------
Epoch 6/10
->Training phase
  Loss=0.9066
  Accuracy=0.69
->Validation phase
  Loss=0.9275
  Accuracy=0.69
-----------------------------------
Epoch 7/10
->Tra

## **Summary**

In [ ]:
import pickle

drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/STL-10/'

with open(drive_path + "stl10_mean_roc_auc_scores.pickle", "rb") as f: auc_scores = pickle.load(f)
with open(drive_path + "stl10_macro_f1_scores.pickle", "rb") as f: f1_scores = pickle.load(f)
with open(drive_path + "stl10_haas_scores.pickle", "rb") as f: haas_scores = pickle.load(f)
with open(drive_path + "stl10_accuracies.pickle", "rb") as f: accuracies = pickle.load(f)

In [ ]:
import pandas as pd

n = [i for i in range(1,17)]
models = ['vgg19_no{}'.format(i) for i in n]

dictionary = {'Model':models,
              'HAAS score':haas_scores,
              'Mean AUC score':[round(score,3) for score in auc_scores],
              'Accuracy':[round(score,3) for score in accuracies],
              'Macro F1':[round(score,3) for score in f1_scores]}

df = pd.DataFrame.from_dict(dictionary)
df.style.set_properties(**{'text-align': 'center'})
df

,Model,HAAS score,Mean AUC score,Accuracy,Macro F1
0,vgg19_no1,1.034,0.964,0.738,0.737
1,vgg19_no2,1.009,0.924,0.546,0.541
2,vgg19_no3,0.996,0.986,0.840,0.840
3,vgg19_no4,1.034,0.966,0.727,0.731
4,vgg19_no5,0.990,0.992,0.883,0.883
5,vgg19_no6,0.998,0.985,0.838,0.840
6,vgg19_no7,0.996,0.986,0.847,0.847
7,vgg19_no8,0.988,0.987,0.849,0.850
8,vgg19_no9,0.994,0.958,0.720,0.721
9,vgg19_no10,0.985,0.978,0.807,0.808
